In [ ]:
#Загрузка DQN модели (Соперник для AlphaZero)

In [431]:
#Разархивирование архива, в котором находится модель DQN
import zipfile 
import os

archive = zipfile.ZipFile('./agents_20000.zip', 'r') # Extract to current directory 
archive.extractall('./dqn')

In [432]:
#Инфа о файле (Проверка, что файл был разархивирован в полном объеме)
os.stat('./dqn/agents_20000.pt')

os.stat_result(st_mode=33188, st_ino=1807524, st_dev=51713, st_nlink=1, st_uid=0, st_gid=0, st_size=1947598459, st_atime=1654906923, st_mtime=1654906929, st_ctime=1654906929)

In [447]:
#from open_spiel.python.algorithms import random_agent
#from open_spiel.python.algorithms import alpha_zero
#!pip install torch

!pip install --upgrade open_spiel
import pyspiel
from open_spiel.python import rl_environment
from open_spiel.python.pytorch import dqn as dqn_pt

In [448]:
game = 'breakthrough'
env = rl_environment.Environment(game)
info_state_size = env.observation_spec()["info_state"][0]
num_actions = env.action_spec()["num_actions"]
bots = [
            dqn_pt.DQN(
            epsilon_start=0,
            player_id=idx,
            state_representation_size=info_state_size,
            num_actions=num_actions,
            hidden_layers_sizes=[int(l) for l in [64, 64]],
            replay_buffer_capacity=int(1e5),
            batch_size=32) for idx in range(2)
        ]
with open('dqn/agents_20000.pt', 'rb') as f:
    bots = pickle.load(f)
agents = bots

In [ ]:
#AZ ходит первым

In [787]:
import pexpect
import torch
import numpy as np
import re
import pickle

child = pexpect.spawn("./open_spiel_repo/build/examples/alpha_zero_torch_game_example \
                      --game=breakthrough \
                      --player1=az \
                      --player2=human \
                      --az_path=BREAKTHROUGH_3 \
                      --az_checkpoint=5")
board_size = 8 
current_player = 1  # Если "мы" ходим вторыми
num_games = 1
verbose = True
players = ['AZ', 'DQN']

In [715]:
# Поиск вражеской позиции (Если "мы" ходим вторыми)
wins = [0, 0]
for k in range(num_games):
    while True:
        board = []

        for i in range(2):
            child.expect_exact('state:\r\n')
            if verbose: print(players[not bool(i)] + ' move:')
            for _ in range(board_size):
                cur_str = child.readline()
                if verbose: 
                    print(cur_str.decode("utf-8"), end='')
                if (current_player == i): 
                    board += cur_str.decode("utf-8")[1:board_size + 1]
            if verbose: 
                print(child.readline().decode("utf-8"))

        #Преобразование доски символов к векторному представлению
        board = np.array(board)
        info_state = torch.Tensor(np.concatenate((board == 'b', board == 'w', board == '.')))
        info_state.sort(dim=-1)
        info_state = torch.unsqueeze(info_state, dim=0)

        #Нажатие на enter (для отображения списка возможных ходов)
        child.sendline('')
        child.sendline('')
        child.expect(['([(s):\r\n]|[Returns:])?'])
        #Получение списка возможных ходов
        cur_str = child.readline().decode("utf-8")
        if (cur_str == '\nReturns: -1, 1\r\n'):
            wins[1] += 1
        if (cur_str == '\nReturns: 1, -1\r\n'):
            wins[0] += 1 
        if (cur_str == '\nReturns: -1, 1\r\n' or cur_str == '\nReturns: 1, -1\r\n' ): 
            break
        moves_str = ''
        while cur_str != 'Choose an action (empty to print legal actions): \r\n':
            moves_str += cur_str
            cur_str = child.readline().decode("utf-8")
        legal_actions = list(map(int, re.findall(r'(\d{1,5}):\s+\w\d\w\d', moves_str)))
        #print(legal_actions)

        # Выбор и имитация хода dqn
        q_values = agents[current_player]._q_network(info_state).detach()[0]
        legal_q_values = q_values[legal_actions]
        action = legal_actions[torch.argmax(legal_q_values)]
        child.sendline(str(action)) # Имитация "нашего" хода
        #print('action: ' + str(action))
print(players)
print(wins)

DQN move:
8bbbbbbbb
7bbbbbbbb
6........
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7bbbb.bbb
6...b....
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7bbbb.bbb
6...b....
5........
4........
3.w......
2w.wwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7bbbb.bbb
6........
5..b.....
4........
3.w......
2w.wwwwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7bbbb.bbb
6........
5..b.....
4..w.....
3........
2w.wwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7bb.b.bbb
6...b....
5..b.....
4..w.....
3........
2w.wwwwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7bb.b.bbb
6...b....
5.wb.....
4........
3........
2w.wwwwww
1wwwwwwww
 abcdefgh

AZ move:
8b.bbbbbb
7bbbb.bbb
6...b....
5.wb.....
4........
3........
2w.wwwwww
1wwwwwwww
 abcdefgh

DQN move:
8b.bbbbbb
7bbbb.bbb
6.w.b....
5..b.....
4........
3........
2w.wwwwww
1wwwwwwww
 abcdefgh

AZ move:
8b.bbbbbb
7bb.b.bbb
6.b.b....
5..b.....
4........
3........
2w.wwwwww
1wwwwwwww
 abcde

In [ ]:
#DQN ходит первым

In [843]:
import pexpect
import torch
import numpy as np
import re
import pickle
import random

child = pexpect.spawn("./open_spiel_repo/build/examples/alpha_zero_torch_game_example \
                      --game=breakthrough \
                      --player1=human \
                      --player2=az \
                      --az_path=BREAKTHROUGH_3 \
                      --az_checkpoint=-1 \
                      --num_games=10")
board_size = 8 
current_player = 0  # Если "мы" ходим вторыми
num_games = 10
verbose = True
players = ['DQN', 'AZ']

In [808]:
# 770 step of model
def state_returns():
        global cur_str
        child.expect('state\:\\r\\n|Returns\:')
        cur_str = child.readline().decode("utf-8")
        if (cur_str == ' -1, 1\r\n'):
            wins[1] += 1
        if (cur_str == ' 1, -1\r\n'):
            wins[0] += 1 
        if (cur_str == ' -1, 1\r\n' or cur_str == ' 1, -1\r\n' ): 
            return True
        return False

# Поиск вражеской позиции (Если "мы" ходим вторыми)
wins = [0, 0]
for k in range(num_games):
    flag_first = True 
    while True:
        board = []
        if (state_returns()): break
        if verbose: print('AZ move:')
        for _ in range(board_size):
            if verbose: 
                print(cur_str, end='')
            board += cur_str[1:board_size + 1]
            cur_str = child.readline().decode("utf-8")
        if verbose: 
            print(cur_str)

        #Преобразование доски символов к векторному представлению
        board = np.array(board)
        info_state = torch.Tensor(np.concatenate((board == 'b', board == 'w', board == '.')))
        info_state = torch.unsqueeze(info_state, dim=0)
        info_state.sort()
        
        #Нажатие на enter (для отображения списка возможных ходов)
        child.sendline('')
        child.sendline('')
        child.expect(['([(s):\r\n]|[Returns:])?'])
        
        #Получение списка возможных ходов
        cur_str = child.readline().decode("utf-8")
        if (cur_str == '\nReturns: -1, 1\r\n'):
            wins[1] += 1
        if (cur_str == '\nReturns: 1, -1\r\n'):
            wins[0] += 1 
        if (cur_str == '\nReturns: -1, 1\r\n' or cur_str == '\nReturns: 1, -1\r\n' ): 
            break
        moves_str = ''
        while cur_str != 'Choose an action (empty to print legal actions): \r\n':
            moves_str += cur_str
            cur_str = child.readline().decode("utf-8")
        legal_actions = list(map(int, re.findall(r'(\d{1,5}):\s+\w\d\w\d', moves_str)))
        print(legal_actions)

        # Выбор и имитация хода dqn
        if (flag_first):
            action = random.sample(legal_actions, 1)[0]
            child.sendline(str(action))
            flag_first = False
        else:
            q_values = agents[current_player]._q_network(info_state).detach()[0]
            legal_q_values = q_values[legal_actions]
            action = legal_actions[torch.argmax(legal_q_values)]
            child.sendline(str(action)) # Имитация "нашего" хода
            #print('action: ' + str(action))
        
        if (state_returns()): break
        if verbose: print('DQN move:')
        for _ in range(board_size):
            if verbose: 
                print(cur_str, end='')
            cur_str = child.readline().decode("utf-8")
        if verbose: 
            print(cur_str)
print(players)
print(wins)

AZ move:
8bbbbbbbb
7bbbbbbbb
6........
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

[98, 112, 132, 146, 160, 180, 100, 120, 134, 148, 168, 182, 108, 122, 136, 156, 170, 110, 124, 144, 158, 172]
DQN move:
8bbbbbbbb
7bbbbbbb.
6......b.
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7bbbbbbb.
6......b.
5........
4........
3w.......
2.wwwwwww
1wwwwwwww
 abcdefgh

[98, 112, 132, 146, 264, 172, 100, 120, 134, 148, 266, 76, 108, 122, 136, 156, 268, 86, 110, 124, 144, 158, 168]
DQN move:
8bbbbbbbb
7.bbbbbb.
6.b....b.
5........
4........
3w.......
2.wwwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7.bbbbbb.
6.b....b.
5........
4........
3w.......
2wwwwwwww
1w.wwwwww
 abcdefgh

[2, 108, 124, 144, 158, 168, 204, 112, 132, 146, 264, 172, 206, 12, 134, 148, 266, 76, 208, 122, 136, 156, 268, 86]
DQN move:
8bbbbbbbb
7.bbbbbb.
6......b.
5..b.....
4........
3w.......
2wwwwwwww
1w.wwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7.bbbbbb.
6......b.
5..b.....
4........

IndexError: argmax(): Expected reduction dim to be specified for input.numel() == 0.

In [844]:
# 1000 step of model
def state_returns():
        global cur_str
        child.expect('state\:\\r\\n|Returns\:')
        cur_str = child.readline().decode("utf-8")
        if (cur_str == ' -1, 1\r\n'):
            wins[1] += 1
        if (cur_str == ' 1, -1\r\n'):
            wins[0] += 1 
        if (cur_str == ' -1, 1\r\n' or cur_str == ' 1, -1\r\n' ): 
            return True
        return False

# Поиск вражеской позиции (Если "мы" ходим вторыми)
wins = [0, 0]
for k in range(num_games):
    flag_first = True 
    while True:
        board = []
        if (state_returns()): break
        if verbose: print('AZ move:')
        for _ in range(board_size):
            if verbose: 
                print(cur_str, end='')
            board += cur_str[1:board_size + 1]
            cur_str = child.readline().decode("utf-8")
        if verbose: 
            print(cur_str)

        #Преобразование доски символов к векторному представлению
        board = np.array(board)
        info_state = torch.Tensor(np.concatenate((board == 'b', board == 'w', board == '.')))
        info_state = torch.unsqueeze(info_state, dim=0)
        info_state.sort()
        
        #Нажатие на enter (для отображения списка возможных ходов)
        child.sendline('')
        child.sendline('')
        child.expect(['([(s):\r\n]|[Returns:])?'])
        
        #Получение списка возможных ходов
        cur_str = child.readline().decode("utf-8")
        if (cur_str == '\nReturns: -1, 1\r\n'):
            wins[1] += 1
        if (cur_str == '\nReturns: 1, -1\r\n'):
            wins[0] += 1 
        if (cur_str == '\nReturns: -1, 1\r\n' or cur_str == '\nReturns: 1, -1\r\n' ): 
            break
        moves_str = ''
        while cur_str != 'Choose an action (empty to print legal actions): \r\n':
            moves_str += cur_str
            cur_str = child.readline().decode("utf-8")
        legal_actions = list(map(int, re.findall(r'(\d{1,5}):\s+\w\d\w\d', moves_str)))

        # Выбор и имитация хода dqn
        if (flag_first):
            action = random.sample(legal_actions, 1)[0]
            child.sendline(str(action))
            flag_first = False
        else:
            q_values = agents[current_player]._q_network(info_state).detach()[0]
            legal_q_values = q_values[legal_actions]
            action = legal_actions[torch.argmax(legal_q_values)]
            child.sendline(str(action)) # Имитация "нашего" хода
            #print('action: ' + str(action))
        
        if (state_returns()): break
        if verbose: print('DQN move:')
        for _ in range(board_size):
            if verbose: 
                print(cur_str, end='')
            cur_str = child.readline().decode("utf-8")
        if verbose: 
            print(cur_str)
print(players)
print(wins)

AZ move:
8bbbbbbbb
7bbbbbbbb
6........
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7bbbbb.bb
6.....b..
5........
4........
3........
2wwwwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7bbbbb.bb
6.....b..
5........
4........
3.w......
2w.wwwwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7.bbbb.bb
6.b...b..
5........
4........
3.w......
2w.wwwwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7.bbbb.bb
6.b...b..
5........
4........
3.w..w...
2w.w.wwww
1wwwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7.bbbb.bb
6.....b..
5..b.....
4........
3.w..w...
2w.w.wwww
1wwwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7.bbbb.bb
6.....b..
5..b.....
4........
3.w..w...
2www.wwww
1.wwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7.bbbb.bb
6.....b..
5........
4...b....
3.w..w...
2www.wwww
1.wwwwwww
 abcdefgh

AZ move:
8bbbbbbbb
7.bbbb.bb
6.....b..
5........
4...b....
3.w..ww..
2www..www
1.wwwwwww
 abcdefgh

DQN move:
8bbbbbbbb
7.bbbb.bb
6.....b..
5........
4........
3.wb.ww..
2www..www
1.wwwwwww
 abcde

In [ ]:
#help

In [585]:
! ./open_spiel_repo/build/examples/alpha_zero_torch_game_example --help

alpha_zero_torch_game_example: Warning: SetProgramUsageMessage() never called

  Flags from repo/open_spiel/examples/alpha_zero_torch_game_example.cc:
    --az_batch_size (Batch size of AZ inference.); default: 1;
    --az_cache_shards (Cache shards of AZ algorithm.); default: 1;
    --az_cache_size (Cache size of AZ algorithm.); default: 16384;
    --az_checkpoint (Checkpoint of AZ model.); default: -1;
    --az_graph_def (AZ graph definition file name.); default: "vpnet.pb";
    --az_path (Path to AZ experiment.); default: "";
    --az_threads (Number of threads to run for AZ inference.); default: 1;
    --game (The name of the game to play.); default: "tic_tac_toe";
    --max_memory_mb (The maximum memory used before cutting the search short.);
      default: 1000;
    --max_simulations (How many simulations to run.); default: 10000;
    --num_games (How many games to play.); default: 1;
    --player1 (Who controls player1.); default: "az";
    --player2 (Who controls player2.); def